In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from time import sleep

<h1>Finding 200 top coins and creating first table</h1>


In [2]:
def HtmlParser(url):
    import requests
    from bs4 import BeautifulSoup
    page = requests.get(url)
    bs = BeautifulSoup(page.content, 'html.parser')
    return bs

In [3]:
#finding the coins on the specified day by url

soup = HtmlParser("https://coinmarketcap.com/historical/20230825/")
soup.body.prettify()
#all the rows of page
AllRows = soup.find_all('tr')
MainLinks = []
Titles = []
for i in range(3 , 203):
  cur = AllRows[i].find('a')
    #first link allways is coin main link
  MainLinks.append(f"https://coinmarketcap.com{cur['href']}")
    #some coins have title tag and for others which don't the title is written on first content
  try:
    Titles.append(cur['title'])
  except:
    Titles.append(cur.contents[0])
SymBols = []
HystLinks = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
    #finding symbol based on html pattern
  SymBols.append(bs.find('span' , {'data-role':"coin-symbol"}).contents[0])
  txt = str(bs)
  pos = txt.find("See historical")
    #search for historical data on the entire content of the page
  LinkTxt = txt[pos-200:pos]
  pos2 = LinkTxt.find("href")
    #the link attached to "historical data is written somewhere before
    #200 chars is enough and 100 chars is not for example
  Hlink = LinkTxt[pos2 + 6:]
  Hlink = Hlink[ : Hlink.find(" ") - 1]
    #historical data link starts exactly 6 chars after href and ends after " and an space
  HystLinks.append( f"https://coinmarketcap.com{Hlink}" )

In [4]:
GitHubLinks = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
  txt = str(bs)
   #search for github in page
  pos = txt.find("github.com")
  txt2 = txt[pos:]
    #some links end in ) and some in "
  pos2 = txt2.find('"')
  pos3 = txt2.find(')')
  pos2 = min(pos2 , pos3)
  gitlink = txt2[:pos2]
  GitHubLinks.append(gitlink)

In [5]:
df = pd.DataFrame(list(zip(Titles, SymBols , MainLinks , HystLinks , GitHubLinks )),
               columns =['Name', 'Symbol' , 'MainLink' , 'HistoricalLink' , 'GitHubLink'])
df.to_csv("FirstTable.csv")

In [ ]:
Tags = []
for i in range(0 , 200):
  url = MainLinks[i]
  bs = HtmlParser(url)
    #find tags based on html
  temp = bs.find_all('div' , {'class' : "sc-16891c57-0 sc-b7faf77f-0 jaPKUl"})
  taglist = []
    # no tag
  if len(temp) < 1:
    Tags.append(taglist)
    continue
  for x in temp[0]:
    taglist.append(x.text)
    #the last one is not a tag name and it's common in all coins
  taglist.pop()
  Tags.append(taglist)

<h1>finding last year data for each coin based on crawled historical links</h1>

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver, 10)
sleeptime = 1

In [ ]:
#reading data from pre created csv from previous part dataframe to don't run the previous part every time
df = pd.read_csv("FirstTable.csv")
df = df['HistoricalLink']

In [ ]:
for url in df:
    driver.get(url)
    #click on calender
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[1]'))).click()
    #go to selecting year page
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    #select start year
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/span[6]'))).click()
    #select start month
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/span[8]'))).click()
    #select start day
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[2]/div[4]/div[5]'))).click()
    #go to selecting year page
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]'))).click()
    #select end year
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[2]/span[8]'))).click()
    #select end month
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[1]/div[1]/div[1]/span[8]'))).click()
    #select end day
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[1]/div[1]/div/div/div[2]/div[2]/div[4]/div[6]'))).click()
    wait.until(EC.element_to_be_clickable((By.XPATH, '//*[@id="tippy-1"]/div/div[1]/div/div/div[2]/span/button'))).click()
    #waiting for the site to load csv and then wait for complete download
    sleep(sleeptime*5)
    wait.until(EC.presence_of_element_located((By.XPATH, '//*[@id="__next"]/div[2]/div[1]/div[2]/div/div/div/div[2]/div/div[1]/div/button[2]'))).click()
    sleep(sleeptime*5)

NoSuchWindowException: Message: no such window: target window already closed
from unknown error: web view not found
  (Session info: chrome=116.0.5845.180)
Stacktrace:
	GetHandleVerifier [0x00007FF6066352A2+57122]
	(No symbol) [0x00007FF6065AEA92]
	(No symbol) [0x00007FF60647E3AB]
	(No symbol) [0x00007FF60645E1B9]
	(No symbol) [0x00007FF6064DB417]
	(No symbol) [0x00007FF6064EE24F]
	(No symbol) [0x00007FF6064D6DB3]
	(No symbol) [0x00007FF6064AD2B1]
	(No symbol) [0x00007FF6064AE494]
	GetHandleVerifier [0x00007FF6068DEF82+2849794]
	GetHandleVerifier [0x00007FF606931D24+3189156]
	GetHandleVerifier [0x00007FF60692ACAF+3160367]
	GetHandleVerifier [0x00007FF6066C6D06+653702]
	(No symbol) [0x00007FF6065BA208]
	(No symbol) [0x00007FF6065B62C4]
	(No symbol) [0x00007FF6065B63F6]
	(No symbol) [0x00007FF6065A67A3]
	BaseThreadInitThunk [0x00007FFDC30F7C24+20]
	RtlUserThreadStart [0x00007FFDC384D721+33]
